<a href="https://colab.research.google.com/github/vidakpop/livewstock-lumpyskin-ml/blob/main/lymph_skin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!wget https://github.com/vidakpop/livewstock-lumpyskin-ml/archive/refs/heads/main.zip
!unzip main.zip -d dataset



--2025-03-01 18:55:40--  https://github.com/vidakpop/livewstock-lumpyskin-ml/archive/refs/heads/main.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/vidakpop/livewstock-lumpyskin-ml/zip/refs/heads/main [following]
--2025-03-01 18:55:40--  https://codeload.github.com/vidakpop/livewstock-lumpyskin-ml/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 20.27.177.114
Connecting to codeload.github.com (codeload.github.com)|20.27.177.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [       <=>          ] 154.35M   466KB/s    in 2m 51s  

2025-03-01 18:58:32 (925 KB/s) - ‘main.zip’ saved [161845855]

Archive:  main.zip
4c099d1bd32400173c98f238ba966e6702b5a9af
   creating: dataset/livewstock-lumpyskin-ml

In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

In [6]:
# Define dataset path
dataset_dir = "/content/dataset/livewstock-lumpyskin-ml-main/archive/Lumpy Skin Images Dataset"  # Main dataset directory containing two folders

In [7]:
dataset_dir

'/content/dataset/livewstock-lumpyskin-ml-main/archive/Lumpy Skin Images Dataset'

In [8]:
# prompt: dispaly the folders and images in dataset_dir

import os

for dirname, _, filenames in os.walk(dataset_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))
    for subdirname in os.listdir(dirname):
        if os.path.isdir(os.path.join(dirname, subdirname)):
            print(os.path.join(dirname, subdirname))


/content/dataset/livewstock-lumpyskin-ml-main/archive/Lumpy Skin Images Dataset/Normal Skin
/content/dataset/livewstock-lumpyskin-ml-main/archive/Lumpy Skin Images Dataset/Lumpy Skin
/content/dataset/livewstock-lumpyskin-ml-main/archive/Lumpy Skin Images Dataset/Normal Skin/Normal_Skin_247.png
/content/dataset/livewstock-lumpyskin-ml-main/archive/Lumpy Skin Images Dataset/Normal Skin/Normal_Skin_65.png
/content/dataset/livewstock-lumpyskin-ml-main/archive/Lumpy Skin Images Dataset/Normal Skin/Normal_Skin_164.png
/content/dataset/livewstock-lumpyskin-ml-main/archive/Lumpy Skin Images Dataset/Normal Skin/Normal_Skin_678.png
/content/dataset/livewstock-lumpyskin-ml-main/archive/Lumpy Skin Images Dataset/Normal Skin/Normal_Skin_290.png
/content/dataset/livewstock-lumpyskin-ml-main/archive/Lumpy Skin Images Dataset/Normal Skin/Normal_Skin_710.png
/content/dataset/livewstock-lumpyskin-ml-main/archive/Lumpy Skin Images Dataset/Normal Skin/Normal_Skin_223.png
/content/dataset/livewstock-lumpys

In [9]:
# Image Preprocessing and Augmentation
img_size = (224, 224)  # Resize all images to 224x224
batch_size = 32


In [10]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80% train, 20% validation
)

In [11]:
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

Found 820 images belonging to 2 classes.


In [12]:
val_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 204 images belonging to 2 classes.


In [13]:
# Build the CNN Model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
# Compile the Model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [15]:
# Train the Model
epochs = 10
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 114s 4s/step - accuracy: 0.5720 - loss: 1.1563 - val_accuracy: 0.6863 - val_loss: 0.5733
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 110s 4s/step - accuracy: 0.6759 - loss: 0.5835 - val_accuracy: 0.6863 - val_loss: 0.6030
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 109s 4s/step - accuracy: 0.6806 - loss: 0.5803 - val_accuracy: 0.6863 - val_loss: 0.6041
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 108s 4s/step - accuracy: 0.6795 - loss: 0.5658 - val_accuracy: 0.7059 - val_loss: 0.5159
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 142s 4s/step - accuracy: 0.7392 - loss: 0.5734 - val_accuracy: 0.7402 - val_loss: 0.5738
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 108s 4s/step - accuracy: 0.7576 - loss: 0.5271 - val_accuracy: 0.7451 - val_loss: 0.4869
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 108s 4s/step - accuracy: 0.7669 - loss: 0.4807 - val_accuracy: 0.7794 - val_loss: 0.4854
Epoch 8/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 109s 4s/step - accuracy: 0.7601 - loss: 0.4881 - val_accuracy: 0.7647 - v

In [17]:
# Load Pretrained MobileNetV2 Model (without top layers)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze pretrained layers


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [18]:
# Add Custom Layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Convert to 1D feature vector
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)  # Prevent overfitting
output_layer = Dense(1, activation='sigmoid')(x)  # Binary classification


In [20]:
# Define new model
model = Model(inputs=base_model.input, outputs=output_layer)



In [21]:
# Compile the Model
model.compile(optimizer=Adam(learning_rate=0.0001),  # Lower LR for fine-tuning
              loss='binary_crossentropy',
              metrics=['accuracy'])



In [22]:
# Train the Model with Early Stopping
epochs = 15
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(train_generator, epochs=epochs, validation_data=val_generator, callbacks=[early_stopping])


Epoch 1/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step - accuracy: 0.6109 - loss: 0.7465 - val_accuracy: 0.7941 - val_loss: 0.4464
Epoch 2/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.7462 - loss: 0.5054 - val_accuracy: 0.8725 - val_loss: 0.3747
Epoch 3/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.8053 - loss: 0.4539 - val_accuracy: 0.8480 - val_loss: 0.3414
Epoch 4/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.8073 - loss: 0.4223 - val_accuracy: 0.9020 - val_loss: 0.3097
Epoch 5/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 67s 3s/step - accuracy: 0.8407 - loss: 0.3793 - val_accuracy: 0.8922 - val_loss: 0.2796
Epoch 6/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 67s 3s/step - accuracy: 0.8694 - loss: 0.3598 - val_accuracy: 0.8725 - val_loss: 0.3029
Epoch 7/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 66s 3s/step - accuracy: 0.8644 - loss: 0.3366 - val_accuracy: 0.8922 - val_loss: 0.2816
Epoch 8/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.8612 - loss: 0.3486 - val_accuracy: 0.8971 - val_loss:

In [23]:
# Save the Model
model.save("lumpy_skin_cattle_detector_transfer.h5")

print("Model training complete and saved!")

Model training complete and saved!


#Testing the model with user input

In [24]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.models import load_model

In [25]:
# Load the trained model
model = load_model("lumpy_skin_cattle_detector_transfer.h5")
